[Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168.pdf)

[Самоучитель клингонского](https://habr.com/ru/company/antiplagiat/blog/507848/)

[Word vectors for 157 languages](https://fasttext.cc/docs/en/crawl-vectors.html)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/embeddings

/content/gdrive/MyDrive/embeddings


## Data

In [3]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

### Загрузка эмбеддингов

In [ ]:
#!unzip cc.uk.300.vec.zip

In [5]:
#!unzip cc.ru.300.vec.zip

In [6]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [7]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [8]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [9]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [10]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.2230587601661682),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872764825820923),
 ('конструктивен', 0.21796566247940063)]

### Загрузка размеченных датасетов

In [11]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [12]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [13]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Маппинг эмбеддингов

Пусть $x_i \in \mathrm{R}^d$ векторное представление слова $i$ на исходном языке, и $y_i \in \mathrm{R}^d$ векторное представление его перевода на целевой язык. Наша цель - выучить линейную трансформацию $W$которая минимизирует Евклидово расстояние между $Wx_i$ и $y_i$ для некоторого подмножества векторов. Все это - Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
или
$$W^*= \arg\min_W ||WX - Y||_F$$

где $||*||_F$ - норма Фробениуса.

В греческой мифологии Прокруст был разбойником, который обитал в находящемся в Аттике Коридалле  и подстерегал путников на пути между Мегарой и Афинами. Он обманом заманивал путешественников в свой дом, укладывал их на своё ложе и тем, кому оно было коротко, отрубал ноги, а кому было длинно — вытягивал ноги по длине этого ложа.
Мы делаем то же самое с нашим пространством :) Прокрустово ложе в данном случае — это целевое пространство для эмбеддингов.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

Обучим маппинг

In [14]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

Посмотрим, что теперь получилось

In [15]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311846256256104),
 ('ноябрь', 0.8278924226760864),
 ('июль', 0.823452889919281),
 ('август', 0.8120500445365906),
 ('декабрь', 0.8039003610610962)]

Метрика - precision@k

In [16]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        result = ru_emb.most_similar(mapped_vectors[i, :].reshape(1, -1))
        if ru in [item[0] for item in result[:topn]]:
          num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val


In [17]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [18]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top1

0.6356589147286822

## Улучшение (orthogonal Procrustean problem)

В оригинальной статье показано, что линейный маппинг должен быть ортогональным. 
Наложим ограничение ортогональности на $W$. Тогда

$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

$$I \text{- identity matrix}$$

Вместо того, чтобы снова решать задачу регрессии, мы можем найти оптимальное ортогональное преобразование, используя SVD.
$$X^TY=U\Sigma V^T\text{, SVD}$$
$$W^*=UV^T$$

In [19]:
from scipy import linalg

In [20]:
def learn_transform(X_train, Y_train):
    matrix = X_train.T @ Y_train
    U, s, V = linalg.svd(matrix, full_matrices=True)
    W = U @ V
    return W

In [21]:
W = learn_transform(X_train, Y_train)

In [22]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.80497145652771),
 ('март', 0.802565336227417),
 ('июнь', 0.8021843433380127),
 ('октябрь', 0.8001737594604492),
 ('ноябрь', 0.7934484481811523),
 ('февраль', 0.7914121747016907),
 ('июль', 0.7908109426498413),
 ('август', 0.7891018390655518),
 ('декабрь', 0.7686372995376587)]

In [29]:
precision(uk_ru_test, np.matmul(X_test, W))

0.6537467700258398

## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [24]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [25]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [26]:
def translate(sentence):
    result = []
    for word in word_tokenize(sentence):
      if word in uk_emb.vocab:
        predicted = ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0]
        result.append(predicted)
    return ' '.join(result)

In [27]:
assert translate(".") == "."
assert translate("1, 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [28]:
for sentence in uk_sentences[:10]:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк –

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала избушку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня , чтобы . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со воскресеньем ... мне огня , мной тебе .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , — – сестричка . садись немного , пока мной пирожки со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала сміттячка, стулила та й біжить.
dst: а 